# Projet 5MA-OSI : Transfert d'organes en domino sous incertitudes

## Introduction

Malgré l'augmentation croissante du nombre de transplantations d'organes effectuées chaque année (environ 6000 en 2017 dont 3782 transplantations de reins), la demande reste en perpétuelle augmentation. Ainsi 6000 organes, dont 3782 reins, ont été transplantés en 2017, mais il y avait encore 24000 personnes en attente d'un organe la même année. Les organes transplantés peuvent provenir d'un donneur décédé ou, dans le cas des reins et du foie, d'un donneur vivant consentant, le plus souvent membre de la famille du patient. Hélas, même si un proche accepte de prendre ce risque pour sa santé, il ne sera pas forcément compatible avec le patient. Pour cette raison, les pratiques médicales et les législations évoluent dans de nombreux pays afin de permettre la mise en place d'un programme d'échange de dons d'organes.

L'exemple le plus simple d'échange de don d'organes est celui où deux patients $P_1$ et $P_2$ sont accompagnés de donneurs $D_1$ et $D_2$. Les patients sont supposés incompatibles avec les donneurs qui les accompagnent, mais on suppose que $D_1$ est compatible avec $P_2$ et $D_2$ avec $P_1$. Il est alors possible de transplanter un organe de $D_1$ vers $P_2$ et de $D_2$ vers $P_1$ avec le consentement de tous et en suivant la procédure légale.

Plus généralement, un cycle d'échange d'organes associe $k$ paires de patient-donneur $(P_{i_1},D_{i_1}), \dots, (P_{i_k},D_{i_k})$ de sorte que $D_{i_l}$ donne à $P_{i_{l+1}}$ pour $l=1,\dots,k-1$ et $D_{i_k}$ donne à $P_{i_1}$.
Par ailleurs, le point essentiel est que les transferts soient tous réalisés en même temps et dans le même hôpital pour éviter qu'une rétractation de dernière minute ne lèse un patient et son donneur, et que les patients et donneurs venus ensemble et leur famille puissent se soutenir émotionnellement durant l'hospitalisation. 
Pour cette raison, le nombre d'échanges prenant place au sein d'un même cycle est nécessairement limité. En pratique, l'organisation d'un cycle de trois paires est déjà une épreuve pour le personnel d'un hôpital, et le plus grand cycle ayant jamais eu lieu a a impliqué six patients et donneurs.

Dans ce projet, nous prendrons le point de vue de l'organisme national responsable de la gestion du programme d'échange d'organes. 
À chaque phase d'échange, l'objectif de cet organisme est de choisir un ensemble de cycles d'échanges entre paires compatibles afin de maximiser le nombre de patients recevant un organe. Dans certains cas, on peut aussi donner une priorité à certains patients en fonction de la gravité de leur état ou de la durée de leur attente. 
Pour cela, on pourra attribuer des poids différents à chaque patient et maximiser la somme des poids des patients recevant un organe. 

## Étude du problème déterministe


### Jeux de données

Tous vos tests seront basés sur les jeux de données de la [PrefLib](https://www.preflib.org/dataset/00036). Ces jeux de données ne correspondent pas à des programmes d'échanges d'organes réels pour des raisons de confidentialité, mais ils reproduisent la structure des données réelles plus fidèlement que les données aléatoires utilisées pour le projet de RO. Ils 
sont constituées d'informations individuelles et d'un graphe de compatibilité. Chaque fichier .wmd décrit un graphe de compatibilité orienté, $G=(V,A)$, où chaque sommet de $V$ représente une paire patient-donneur et où un arc entre deux paires $(P_k,D_k)$ et $(P_l,D_l)$ signifie que $D_k$ est compatible avec $P_l$. La compatibilité est obtenue à partir des données biologiques individuelles (e.g., les groupes sanguins) et d'un _test croisé_ lors duquel des biologistes mettent en présence des tissus d'un malade et d'un donneur supposé. Chaque fichier est composé comme suit :  
- les 11 premières ligne contiennent diverses informations sur le fichier, dont le nombre de sommets ($n$) et le nombre d'arcs ($m$),
- les $n$ lignes suivantes nomment les sommets (un sommet par paire),
- les $m$ lignes restantes contiennent les arcs du graphe et le poids de chaque arc. Le poids de chaque arc indique l'urgence de la situation du malade de la paire destination. 

Je fournis ci-dessous une fonction permettant de lire les fichiers .wmd et retournant un graphe de compatibilité pondéré par les poids de chaque patient. Pour vous aider à vous lancer, je fournis déjà quelques instances en accompagnement de ce sujet sur Moodle. Elles contiennent un nombre croissant de paires patient-donneur allant de 16 à 512.

In [1]:
# Include the necessary packages
using Random, Graphs, JuMP, HiGHS, DelimitedFiles, Distributions

In [2]:
# Exemple d'inclusion d'un notebook : tout le notebook est exécuté
# using NBInclude
# @nbinclude("KEP_readfile.ipynb")

# Exemple en incluant directement un fichier Julia : ici, j'ai ajouté les lignes de test dans le fichier .jl afin d'illustrer que les sorties du fichier sont redirigés vers ce notebook au moment où il est inclus
include("KEP_readfile.jl")

read_wmd_file

In [3]:
# Exemple d'utilisation de la fonction supposant que les fichiers sont dans le dossier data_KEP
G, edge_weight = read_wmd_file("data_KEP/KEP_001.wmd");
println("Number of vertices: ", nv(G));
println("Number of arcs: ", ne(G));
println("List of arcs with weights:")
for e in edges(G)
    print("($(e.src),$(e.dst)):$(edge_weight[e.src,e.dst]), ");
end

Number of vertices: 16
Number of arcs: 59
List of arcs with weights:
(1,5):1.0, (1,6):1.0, (2,1):1.0, (2,3):1.0, (2,11):1.0, (2,12):1.0, (3,5):1.0, (3,8):1.0, (4,1):1.0, (4,3):1.0, (4,11):1.0, (6,1):1.0, (6,3):1.0, (6,11):1.0, (7,1):1.0, (7,3):1.0, (7,11):1.0, (7,12):1.0, (8,1):1.0, (8,3):1.0, (8,11):1.0, (9,1):1.0, (9,3):1.0, (9,11):1.0, (9,12):1.0, (10,1):1.0, (10,2):1.0, (10,3):1.0, (10,5):1.0, (10,8):1.0, (10,9):1.0, (10,11):1.0, (10,12):1.0, (10,14):1.0, (10,15):1.0, (10,16):1.0, (12,1):1.0, (12,3):1.0, (12,11):1.0, (13,2):1.0, (13,3):1.0, (13,5):1.0, (13,7):1.0, (13,8):1.0, (13,9):1.0, (13,11):1.0, (13,14):1.0, (13,15):1.0, (13,16):1.0, (14,1):1.0, (14,3):1.0, (14,11):1.0, (14,12):1.0, (15,1):1.0, (15,3):1.0, (15,11):1.0, (16,5):1.0, (16,6):1.0, (16,8):1.0, 

### Formulations compactes

Dans le cadre du TD/TP du cours de Recherche Opérationnelle, il vous a été demandé de coder trois formulations PLNE, à savoir :
- celle sans contrainte sur la longueur des cycles
- celle avec des cycles de taille 2
- celle avec la décomposition par hôpital pour des cycles de taille 2 à 6.

J'ai recopié la correction distribuée sur Moodle dans le fichier formulation_compactes.jl. Je l'inclus ci dessous. Afin de prendre le sujet en main, comparer les solutions des 3 modèles sur un ensemble de jeux de données bien choisis. Analyser les résultats d'un point de vue informatique, pratique et sociétal.

In [4]:
# inclusion des fonctions de résolution des formulations compactes
include("formulations_compactes.jl")

# tests de bon fonctionnement sur le petit graphe importé plus haut
solve_cycle_infini(G, edge_weight);
solve_cycle_2(G, edge_weight);
solve_cycle_K(G, edge_weight, 4);


-------------------------------
Résolution du modèle sans contrainte de longueur de cycle

Statut de la résolution : OPTIMAL
Durée de la résolution : 0.132287
Nombre de transferts réalisés : 4.0
Liste des transferts réalisés :
1 -> 6 ; 3 -> 8 ; 6 -> 1 ; 8 -> 3 ; 
-------------------------------


-------------------------------
Résolution du modèle avec cycles de taille 2

Statut de la résolution : OPTIMAL
Durée de la résolution : 0.000379083
Nombre de transferts réalisés : 4.0
Liste des transferts réalisés :
1 <-> 6 ; 3 <-> 8 ; 
-------------------------------


-------------------------------
Résolution du modèle avec cycle de taille au plus 4

Statut de la résolution : OPTIMAL
Durée de la résolution : 0.023070209
Nombre de transferts réalisés : 4
Liste des transferts réalisés :
1 -> 6 ; 3 -> 8 ; 6 -> 1 ; 8 -> 3 ; 
-------------------------------



### Génération de colonnes

L'objectif de cette partie du projet est de coder une méthode de génération de colonnes pour le problème de dons d'organes en dominos (KEP pour Kidney Exchange Problem en anglais) sans incertitudes. Votre code s'appuiera sur les éléments vus en CM et leur application au KEP vue en TD. Pour compléter cette présentation, [une page de la documentation de JuMP](https://jump.dev/JuMP.jl/stable/tutorials/algorithms/cutting_stock_column_generation/) est dédiée à la génération de colonnes. Elle vous offre une autre entrée en matière sur la question.

1. Coder une fonction résolvant le modèle par cycles après énumération de touts les cycles de taille $K$ ou moins. Tester pour $K=2,3,4$.
2. Coder une méthode de génération de colonnes pour résoudre la relaxation linéaire de la formulation par cycles. Vous implémenterez deux versions de la fonction de _pricing_ (résolution du sous-problème) : une ou le sous-problème est formulé à l'aide d'un ensemble de PLNE, et une ou le sous-problème est résolu par un algorithme de plus long chemin.
3. Coder une fonction qui résout le problème de dons en dominos de façon approchée à l'aide de l'heuristique de génération de colonnes décrite en CM.


In [ ]:
# TO DO : explore Deapth Search / Wide search ? 

function enumerate_cycles(G::SimpleDiGraph, K::Int64)

    cycles = Vector{Vector{Int64}}()
    c = Vector{Int64}(undef, K) # current cycle so we don't reallocate memory each time we add a vertex

    @inbounds for v in vertices(G)
        c[1] = v
        search_cycles!(G,K,cycles,c,1)
    end
    return cycles
end

# Recursive function
function search_cycles!(G::SimpleDiGraph, K::Int64, cycles::Vector{Vector{Int64}}, c::Vector{Int64}, i::Int64)
    # neighborhood exploration
    for ngb in outneighbors(G,c[i])
        if ngb == c[1]
            push!(cycles,c[1:i])  # There is a cycle
        elseif i < K && ngb > c[1] && (K<4 || !detect_subcycles(ngb,c,i))   # No sub-cycles for K < 4
            c[i+1] = ngb
            search_cycles!(G,K,cycles,c,i+1)
        end
    end
end

function detect_subcycles(v::Int64, c::Vector{Int64}, i::Int64)::Bool
    for j in 2:i
        if c[j] == v 
            return true
        end
    end
    return false
end

function brut_force(G::SimpleDiGraph, K::Int64)
    C_K = enumerate_cycles(G,K)
    for c in C_K
        print("\n*** Cycle : \n\t $(c[1])")
        for v in 2:length(c) 
            print(" -> $v")
        end
    end

    p = length(C_K)
    model = Model(HiGHS.Optimizer);
    L = length.(C_K) # cycles lengths
    I = vertices(G) # vertex indices

    
    C_K_i = Vector{Vector{Int64}}(undef, length(I))
    fill!(C_K_i,Vector{Int64}())

    @variable(model, α[i in range(p)], Bin);
    @objective(model, Min, sum(α[c]*l[c] for c in C_K));
    @constraint(model, [i in I], sum(α[c] for c in C_K));


end

brut_force (generic function with 2 methods)

In [ ]:
I = 4


@show(C_K_i[1])

C_K_i[1] = Int64[]


Int64[]

### Expériences numériques

#### Protocole de test

Décrire un  protocole de test permettant l'évaluation du travail fait plus haut. 

1. Dites d'abord à quelles questions vous souhaitez répondre.
2. Décrivez ensuite votre benchmark en justifiant sa pertinence.
3. Annoncez les tests que vous comptez faire et expliquez comment ces tests peuvent répondre à vos questions.

#### Résultats et discussion

Une fois le protocole défini et les tests réalisés vient la phase qui démontre l'utilité de tout le travail fourni jusqu'ici, celle de l'exploitation de vos résultats. En premier lieu, il faut parvenir à présenter de façon compacte, claire et complète les résultats qui vous permettront de tirer des conclusions utiles. Beaucoup de choses peuvent sembler de l'ordre du détail à cette étape, mais elles font vite la différence entre des résultats lisibles et d'autres incompréhensibles. Je rappelle par exemple que :
- le choix des couleurs importe sur les graphes
- tous les graphes sont accompagnés de titre, des informations utiles sur les axes des abscisses et ordonnées, et de ce à quoi correspondent les différentes courbes s'il y en a plusieurs
- l'échelle choisie fera plus ou moins bien apparaître les conclusions que l'on souhaite tirer (parfois les mauvais résultats d'une méthode écraseront totalement les différences entre d'autres méthodes testées)
- tout tableau, figure ou graphe apparaissant dans un document doit être clairement référencée dans le corps du texte (c'est aussi vrai pour les références bibliographiques)
- lors du CM sur les protocoles de tests, je vous présenterai les profil de performance ; ils s'avèrent souvent utiles lorsque l'on compare plus de deux méthodes différentes, que les méthodes ne résolvent pas toutes les instances dans la limite de temps ou que de grandes variations des temps de calcul apparaissent. Un package Julia permet de les tracer : [BenchmarkProfiles](https://jso.dev/BenchmarkProfiles.jl/stable/).
- parfois un tableau ou une figure n'apporte pas grand chose et il peut suffire d'indiquer une moyenne ou un autre agrégat important dans le texte 

Finalement, chaque tableau/graphe/figure doit être décrit et commenté dans votre rapport. L'objectif est de répondre de façon argumentée aux questions essentielles que vous aurez sélectionné au moment de la définition du protocole. Si au final les résultats ne sont pas suffisants pour répondre à ces questions, il faut pouvoir identifier les tests qui auraient pu être utiles, puis soit faire ces nouveaux tests, soit les laisser en perspective en fonction de l'ensemble des questions auxquelles vous avez déjà répondu, de votre intérêt pour le problème et du temps dont vous disposez. Si aucune des méthodes n'est totalement satisfaisante du point de vue du problème traité, indiquez, si vous le pouvez, des méthodes ou des améliorations qui pourraient être explorées dans le futur. C'est aussi là que vous aurez l'occasion de discuter d'éventuelles précautions à prendre pour l'utilisation en contexte réel d'une telle méthode automatique d'aide à la décision. 

__Si toutes les tâches précédentes sont nécessaires pour en arriver là, il s'agit bien d'une étape essentielle d'un projet de mathématiques appliquées. C'est en la lisant que l'on juge de l'utilité du travail réalisé !__

1. Présentez les résultats de vos tests sous une forme appropriée.
2. Discutez vos résultats et indiquer vos conclusions pour l'ensemble de l'étude du problème déterministe de dons d'organes en dominos.

## Approches avec incertitudes

Dans la pratique, il est important de savoir que les tests croisés permettant de confirmer la compatibilité entre donneurs et malades peuvent être lourds à réaliser pour les malades et qu'ils demandent des ressources importantes auprès des services hospitaliers. Chaque test est précédé de consultations médicales pour le patient et le donneur au cours desquels des tissus sont prélevés, puis des manipulations biologiques doivent être réalisées pour étudier la compatibilité des tissus. En outre, le nombre de tests croisés à réaliser est d'ordre quadratique en fonction du nombre de sommets du graphe de compatibilité. 

Les programmes d'échange sont généralement organisés en deux phases. Lors d'une première phase de planification, l'organisme ne dispose que de données individuelles sur chaque donneur et chaque receveur pour déduire la compatibilité _a priori_ entre donneurs et patients. 
Ces données sont principalement le groupe sanguin et le complexe majeur d’histocompatibilité, aussi appelé système HLA. 
Pour ce projet, nous nous contenterons toutefois de prendre en compte le groupe sanguin. 
Il est par ailleurs possible de prendre en compte une probabilité d'échec du test croisé à partir d'autres données individuelles agrégées sous l'indicateur du pourcentage d'anti-corps réactifs (PRA) du malade de chaque paire. 

Mais, les opérations de transfert d'organe ne sont jamais planifiées avant d'avoir confirmé la compatibilité entre le donneur et le malade par un test croisé. 
Suite à la première phase, l'organisme responsable du programme d'échange planifie donc un ensemble de tests croisés pour vérifier la compatibilité entre certains malades et donneurs. 
Sachant la lourdeur des tests croisés, leur nombre est limité en pratique. 
On pourra pour cela considérer une limite fixe, une limite dépendant du nombre de paires patient-donneur ou bien supposer que les tests ne servent qu'à confirmer la compatibilité après avoir décidé les cycles d'échange entre patients a priori compatibles. 
La seconde phase du programme d'échange consiste à planifier les cycles de transferts d'organes une fois les résultats des tests croisés connus. 

Du point de vue de l'aide à la décision, la première phase de l'organisation des transferts est un problème d'optimisation sous incertitudes. On souhaite décider l'ensemble des tests croisés à effectuer pour maximiser le nombres de dons d'organe alors qu'une partie des données est incertaine. Les données individuelles de groupes sanguins et de PRA sont bien connues (on dit aussi qu'elles sont déterministes), mais la compatibilité entre donneurs et malades est incertaine (elle dépend du résultat d'un test croisé). Ici, nous sommes toutefois dans le cas où les données incertaines sont _stochastiques_ puisque nous supposerons que l'on connaît leur distribution de probabilité.

La suite du projet consistera à coder plusieurs approches d'optimisation sous incertitudes pour résoudre ce nouveau problème, plus proche des conditions réelles d'organisation des programmes d'échange d'organes. Les méthodes considérées seront introduites par des séances de cours permettant de poser leur cadre formel et d'énoncer certaines propriétés théoriques qu'elles satisfont. Ainsi, l'énoncé du travail à réaliser donné ci-dessous est synthétique, mais il sera précisé lors de séances de TD durant lesquelles nous étudierons l'application des méthodes générales au problème qui nous intéresse dans ce projet. Pour autant, les mêmes données seront exploitées dans toutes les méthodes. Je détaille donc dans un premier temps la définition de ces données et leur génération.

### Descriptions des données incertaines

Comme déjà vu dans le projet de l'année dernière, la compatibilité entre un malade et un donneur dépend de deux types d'information. Une partie des informations biologiques individuelles permet d'affirmer sans autre manipulation que certains donneurs sont incompatibles avec certains malades. Ici, il s'agit du groupe sanguin de chaque donneur et de chaque patient. Nous savons par exemple qu'un donneur du groupe A n'est pas compatible avec un patient du groupe B. S'il y a incompatibilité entre les groupes sanguins d'un donneur et d'un malade, on sait donc qu'il ne sera pas nécessaire de réaliser un test croisé entre ces deux personnes. 

Les groupes sanguins nous permettent donc de construire un premier graphe de compatibilité _a priori_, $\tilde G=(V,\tilde A)$ où $V$ correspond à l'ensemble des paires du programme d'échange et où un arc $a\in \tilde A$ entre deux paires $(P_k,D_k)$ et $(P_l,D_l)$ signifie que le groupe sanguin de $D_k$ n'est pas incompatible avec celui de $P_l$. 

Des tests croisés pourront ensuite être réalisés sur des arcs de $\tilde G$ si l'on pense que ces arcs pourraient être impliqués dans un cycle de transferts d'organes. Les données incertaines qui nous intéressent pour la suite correspondent donc au résultat de ces tests croisés dont nous supposons connaître la distribution de probabilité.
Plusieurs modèles d'incertitudes sont classiquement regardés dans la littérature pour modéliser la distribution de probabilité de la réussite des tests croisés. Tous considèrent cependant que la réussite d'un test réalisé sur un arc $a$ suit une loi de Bernouilli de probabilité $1-f_a$ où $f_a$ est une probabilité d'échec donnée. Les différents modèles étudiés dans la littérature peuvent considérer des probabilités d'échec constantes, mais les plus réalistes indiquent une dépendance au PRA des malades.

Dans la suite, je donne d'abord une fonction permettant de lire les données individuelles de chaque paire patient/donneur dans les fichiers .dat de la PrefLib. Sachant que l'on ne peut plus supposer connaître la compatibilité réelle (donc après test croisé) entre donneurs et malades, les informations contenues dans les fichiers .wmd ne sont plus pertinentes dans cette partie. À la place, nous construirons un graphe de compatibilité _a priori_ en nous basant sur les groupes sanguins et nous retiendrons les PRA de chaque malade dans un attribut du sommet correspondant. Vous pourrez adapter cette fonction pour les données générées selon la méthode décrite pour le projet de RO.

Je donne ensuite la fonction permettant de calculer des probabilités d'échec pour tous les arcs de $\tilde G$ en fonction d'un paramètre à choisir dans le tableau DISTRIBUTIONS et du PRA des malades.

In [18]:
"""
    gs_compatible

Renvoie true si un groupe sanguin de donneur est compatible avec celui d'un malade 

# Paramètres
* `gsd::String` : groupe sanguin d'un donneur
* `gsm::String` : groupe sanguin d'un malade
"""
function gs_compatible(gsd::String, gsm::String)
    if (gsd == "O") || (gsm == "AB")
        # O peut donner à {O,A,B,AB}, AB peut recevoir de {O,A,B,AB}
        return true
    elseif (gsd == "A") && (gsm == "A")
        #  A donne à {A,AB}
        return true
    elseif (gsd == "B") && (gsm == "B")
        # B donne à {B,AB}
        return true
    else
        # O ne peut pas recevoir de {A,B,AB}, A ne peut pas recevoir {B,AB}, B ne peut pas recevoir de {A,AB}
        return false;
    end
end

"""
    read_dat_file

Contruct Gtilde from a `.dat` file from PrefLib.

# Parameters
* `dat_file::String` : path of the `.dat` file.
"""
function read_dat_file(dat_file::String)
    isfile(dat_file) || throw(ArgumentError(".dat file not found."))

    # Extraction des données individuelles depuis le fichier .dat
    file = readdlm(dat_file, '\n')
    nb_vertices = length(file)-1
    gsm = Dict{Int64,String}() # groupe sanguin du malade
    gsd = Dict{Int64,String}() # groupe sanguin du donneur
    pra = Dict{Int64,Float64}() # PRA du malade
    for line in file[2:end]
        splitted_line = split(line, ',')
        pair = parse(Int, splitted_line[1])
        gsm[pair] = String(splitted_line[2])
        gsd[pair] = String(splitted_line[3])
        pra[pair] = parse(Float64, splitted_line[5])
    end

    # Construction du graphe de compatibilité a priori ̃G ; on fixe le poids des arcs à 1 par défaut (à modifier dans une autre fonction si souhaité) 
    Gtilde = SimpleDiGraph(nb_vertices, 0)
    edge_weight = zeros(nb_vertices,nb_vertices)
    for u in 1:nb_vertices
        for v in u+1:nb_vertices
            if gs_compatible(gsd[u], gsm[v])
                add_edge!(Gtilde, u, v)
                edge_weight[u,v] = 1
            end
            if gs_compatible(gsd[v], gsm[u])
                add_edge!(Gtilde, v, u)
                edge_weight[v,u] = 1
            end
        end
    end

    return Gtilde, edge_weight, gsm, gsd, pra
end

read_dat_file

In [19]:
# Exemple d'utilisation de la fonction supposant que les fichiers sont dans le dossier data
Gtilde, edge_weight, gsm, gsd, pra = read_dat_file("data_KEP/KEP_001.dat");
println("Number of vertices: ", nv(Gtilde));
println("Number of arcs: ", ne(Gtilde));
println("List of arcs with weights:")
for e in edges(Gtilde)
    print("($(e.src),$(e.dst)):$(edge_weight[e.src,e.dst]), ");
end

Number of vertices: 16
Number of arcs: 74
List of arcs with weights:
(1,5):1.0, (1,6):1.0, (1,8):1.0, (2,1):1.0, (2,3):1.0, (2,11):1.0, (2,12):1.0, (3,5):1.0, (3,6):1.0, (3,8):1.0, (4,1):1.0, (4,3):1.0, (4,11):1.0, (4,12):1.0, (6,1):1.0, (6,3):1.0, (6,11):1.0, (6,12):1.0, (7,1):1.0, (7,3):1.0, (7,11):1.0, (7,12):1.0, (8,1):1.0, (8,3):1.0, (8,11):1.0, (8,12):1.0, (9,1):1.0, (9,3):1.0, (9,11):1.0, (9,12):1.0, (10,1):1.0, (10,2):1.0, (10,3):1.0, (10,4):1.0, (10,5):1.0, (10,6):1.0, (10,7):1.0, (10,8):1.0, (10,9):1.0, (10,11):1.0, (10,12):1.0, (10,13):1.0, (10,14):1.0, (10,15):1.0, (10,16):1.0, (12,1):1.0, (12,3):1.0, (12,11):1.0, (13,1):1.0, (13,2):1.0, (13,3):1.0, (13,4):1.0, (13,5):1.0, (13,6):1.0, (13,7):1.0, (13,8):1.0, (13,9):1.0, (13,10):1.0, (13,11):1.0, (13,12):1.0, (13,14):1.0, (13,15):1.0, (13,16):1.0, (14,1):1.0, (14,3):1.0, (14,11):1.0, (14,12):1.0, (15,1):1.0, (15,3):1.0, (15,11):1.0, (15,12):1.0, (16,5):1.0, (16,6):1.0, (16,8):1.0, 

In [22]:
DISTRIBUTIONS = ["Constant","Binomial","BinomialUNOS","BinomialAPD","NoFailure"]

"""
    get_failure_rates

Generate failure rates on each edge, and add its value as a property to the edge of the kep_graph. 

# Parameters
* `kep_graph::SimpleDiGraph` : graph describing the pairs and compatibilities
* `pra::Dict{Int64,Float64}` : PRA de chaque malade
* `distribution::String` : type of distirbution of uncertainties; to be chosen in the DISTRIBUTIONS vector
"""
function get_failure_rates(kep_graph::SimpleDiGraph, pra::Dict{Int64,Float64}, distribution::String)

    failure_rate = Dict{Tuple{Int64,Int64},Float64}() # probabilité d'échec d'un transfert d'organe du donneur d'une paire vers le patient d'une autre

    for edge in edges(kep_graph)
        # Failure rates depend on the chosen distribution of uncertainties
        if distribution == "Constant"
            # constant failure rates equal to 70%
            failure_rate[edge.src,edge.dst] = 0.7
        elseif distribution == "Binomial"
            if rand() < 0.25
                # random failure rates equal to 10% on average for 25% edges
                failure_rate[edge.src,edge.dst] = rand() * 0.2
            else
                # random failure rates equal to 90% on average for 75% edges
                failure_rate[edge.src,edge.dst] = 0.8 + rand() * 0.2
            end
        elseif distribution == "BinomialUNOS"
            # %pra denotes the panel reactive antibody level
            # %pra of the patient < 0.8 : UNOS low sensitized patients
            if pra[edge.dst] < 0.8
                # failure rate equal to 10% if the patient is low sensitized
                failure_rate[edge.src,edge.dst] = 0.1
            else
                # failure rate equal to 90% otherwise 
                failure_rate[edge.src,edge.dst] = 0.9
            end
        elseif distribution  == "BinomialAPD"
            # %pra denotes the panel reactive antibody level
            # %pra of the patient < 0.75 : APD low sensitized patients
            if pra[edge.dst] < 0.75
                # failure rate equal to 28% if the patient is low sensitized
                failure_rate[edge.src,edge.dst] = 0.28
            else
                # failure rate equal to 58% otherwise 
                failure_rate[edge.src,edge.dst] = 0.58
            end
        elseif distribution == "NoFailure"
            # failure rates equal to 0
            failure_rate[edge.src,edge.dst] = 0.
        end
    end

    return failure_rate
end


get_failure_rates

In [26]:
get_failure_rates(Gtilde, pra, "Binomial");

### Travail à réaliser

_(Cette partie n'est pas à rendre pour le 14 novembre. Elle ne sera commencée qu'après le premier CM de la partie stochastique, fin novembre.)_

Vous savez désormais récupérer des jeux de données pour le problème avec incertitudes à partir de la PrefLib et vous pouvez adapter ce travail pour en générer de nouveaux selon la méthode décrite au projet de 4A. L'objet de la suite de cette partie sera de coder les méthodes d'optimisation sous incertitudes suivantes.


1. Coder un algorithme permettant de simuler le processus réel en deux étapes, dans lequel l'étape de planification est réalisée à l'aide des méthodes développées dans le cadre déterministe.
2. Utiliser ces simulations pour étudier la sensibilité de la planification aux incertitudes en fonction de la taille maximum des cycles. Commentez.
3. En pratique, de nouvelles personnes entre et sortent du programme d'échange en continu. Donc des programmes sont régulièrement calculés et de nouvelles opérations programmés. Simuler un tel processus dynamique sur 6 mois en supposant que de nouvelles opérations peuvent être planifiées toutes les 2 semaines. Commentez.
4. Coder un modèle de maximisation de l'espérance du nombre de transferts à travers un calcul théorique des probabilités de réussite. Comparer l'utilisation de ce modèle avec les modèles déterministes dans le processus dynamique. Commentez.
5. Analyser d'un point de vue sociétal les résultats des différents algorithmes. Si besoin créer des jeux de données qui permettent d'illustrer certains comportements bénéfiques ou problématiques.
6. Sans la coder, dites ce que l'on peut attendre comme apport de la méthode avec recours demandée ci-dessous.

_Hors programme : la fin de l'énoncé ci-dessous ne fait plus partie du travail à réaliser pendant ce cours. Je la laisse, car j'y décris approche plus réaliste du problème de don en dominos._

1. Coder un modèle avec recours dans lequel un ensemble d'arcs à tester est calculé au premier niveau et les cycles à réaliser sont calculés au second niveau en fonction des résultats des tests croisés.
2. Améliorer le modèle précédent et la méthode utilisée pour le résoudre. On pourra essayer de nombreuses possibilités telles que : différentes formulations pour le problème de second niveau (cf la partie déterministe), la méthode L-Shaped, le renforcement du premier niveau par une contrainte de Jensen, le prétraitement du graphe de compatibilité pour éliminer des arcs/sommets "inutiles", la réduction a priori de l'ensemble d'arcs pour éliminer les arcs trop peu probables (penser aux enjeux éthiques), la restriction du problème à des petits cycles, etc.
3. Coder une approches averse aux risques : maximisation de la C-VaR ou optimisation robuste avec ensemble d'incertitudes de Bertsimas ("budgeted uncertainty")

Comme indiqué plus tôt, ces questions seront détaillées lors des séances de CM et de TD. Dans tous les cas, vous devrez définir un protocole de tests pour évaluer ces différentes méthodes et les comparer entre elles. Vous pourrez vous référer au travail effectué sur la partie déterministe pour construire et décrire clairement un protocole rigoureux qui permettra de répondre à un ensemble de questions importantes pour le problème réel. __Ce protocole, les résultats des tests qui en découleront et la discussion qui suivra sont les éléments permettront de montrer votre compréhension du cours et votre capacité à prendre du recul.__